### Modelo de Previsão de Jogos de Loteria com Machine Learning (Random Forest)


In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSrRRnUWXFPgVrQr6VzI_mlBnubdAXIfTZoC0ozAG9y2Ovqw6LNchhAV9L8ZzavXg/pub?output=csv')
#Super 7

In [ ]:
df.head()

,Concurso,Data Sorteio,Coluna 1,Coluna 2,Coluna 3,Coluna 4,Coluna 5,Coluna 6,Coluna 7,Ganhadores 7 acertos,...,Ganhadores 5 acertos,Rateio 5 acertos,Ganhadores 4 acertos,Rateio 4 acertos,Ganhadores 3 acertos,Rateio 3 acertos,Acumulado 7 acertos,Arrecadação Total,Estimativa Prêmio,Observação
0,1,02/10/2020,2,9,9,8,7,7,6,0,...,272,"R$1.066,16",4330,"R$66,97",41129,"R$5,00","R$744.334,69","R$4.934.145,00","R$1.100.000,00",Os valores apresentados no link Locais da Sort...
1,2,05/10/2020,8,8,1,5,6,7,7,0,...,157,"R$809,26",2308,"R$55,04",21215,"R$5,00","R$1.070.457,92","R$2.198.630,00","R$1.400.000,00",Os valores apresentados no link Locais da Sort...
2,3,07/10/2020,7,2,6,6,5,4,4,1,...,227,"R$618,18",2642,"R$53,11",23324,"R$5,00","R$0,00","R$2.427.070,00","R$300.000,00",Os valores apresentados no link Locais da Sort...
3,4,09/10/2020,5,7,8,1,8,2,5,0,...,139,"R$937,52",2031,"R$64,16",18493,"R$5,00","R$334.485,24","R$2.217.397,50","R$700.000,00",Os valores apresentados no link Locais da Sort...
4,5,14/10/2020,6,4,0,1,0,6,9,0,...,132,"R$1.220,23",2288,"R$70,39",22005,"R$5,00","R$747.915,95","R$2.730.875,00","R$1.200.000,00",Os valores apresentados no link Locais da Sort...


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler

# Carregar dados
data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSrRRnUWXFPgVrQr6VzI_mlBnubdAXIfTZoC0ozAG9y2Ovqw6LNchhAV9L8ZzavXg/pub?output=csv')
numbers = data.iloc[:, 2:9].values  # Colunas 1 a 7

# Normalização
scaler = MinMaxScaler(feature_range=(0, 1))
numbers_normalized = scaler.fit_transform(numbers)

# Preparar sequências
n_input = 10  # Janela temporal
generator = TimeseriesGenerator(numbers_normalized, numbers_normalized, length=n_input, batch_size=1)

# Modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, 7)),
    Dense(7)
])
model.compile(optimizer='adam', loss='mse')

# Treinamento
model.fit(generator, epochs=50, verbose=1)

# Gerar previsões
def generate_games(n_games, n_numbers):
    seed = numbers_normalized[-n_input:]
    games = []
    for _ in range(n_games):
        prediction = model.predict(seed.reshape(1, n_input, 7))
        denorm = scaler.inverse_transform(prediction)
        rounded = np.round(denorm).astype(int).clip(0, 9).flatten()
        unique = list(np.unique(rounded))[:n_numbers]  # Garantir números únicos
        games.append(unique)
    return games

# Previsões
games_7 = generate_games(5, 7)
games_9 = generate_games(5, 9)
games_10 = generate_games(5, 10)

# Resultados
print("Jogos com 7 números:", games_7)
print("Jogos com 9 números:", games_9)
print("Jogos com 10 números:", games_10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.1325
Epoch 2/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.1024
Epoch 3/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.1044
Epoch 4/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.1022
Epoch 5/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.1008
Epoch 6/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.1028
Epoch 7/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.1014
Epoch 8/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1024
Epoch 9/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1021
Epoch 10/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0995
Epoch 11/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0998
Epoch 12/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0971
Epoch 13/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0976
Epoch 14/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0963
Epoch 15/50
666/666 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - lo